In [81]:
!pip install gensim
!pip install deap
!pip install scikit-learn

### MAIN_CODE

In [82]:
import math
import time
import numpy as np
import scipy.sparse as sp
from gensim.models import LdaModel
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time, math
import numpy as np
from typing import Callable, Tuple, Dict, List
import time
import numpy as np
from sklearn.decomposition import LatentDirichletAllocation
import time
import math
import random
import numpy as np
from deap import base, creator, tools

In [83]:
def load_bow_pair(train_path: str, val_path: str):
    Xtr = sp.load_npz(train_path).tocsr(copy=False)
    Xva = sp.load_npz(val_path).tocsr(copy=False)
    return Xtr, Xva

In [84]:
Xtr, Xva = load_bow_pair("X_agnews_train_bow.npz", "X_agnews_val_bow.npz")

In [85]:
def lda_blackbox(
    T: int,
    alpha: float,
    eta: float,
    *,
    seed: int = 42,
    max_iter: int = 400,
    batch_size: int = 2048,
    learning_method: str = "online"
):
    if T < 2:
        raise ValueError("T must be >= 2")
    if alpha <= 0 or eta <= 0:
        raise ValueError("alpha and eta must be > 0")
    lda = LatentDirichletAllocation(
        n_components=int(T),
        doc_topic_prior=float(alpha),
        topic_word_prior=float(eta),
        learning_method=learning_method,
        max_iter=int(max_iter),
        batch_size=int(batch_size),
        random_state=int(seed),
        evaluate_every=-1,
        n_jobs=-1
    )
    t0 = time.perf_counter()
    lda.fit(Xtr)
    fit_time = time.perf_counter() - t0
    ppl = float(lda.perplexity(Xva))
    return ppl

In [86]:
try:
    creator.FitnessMin
except Exception:
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
try:
    creator.Individual
except Exception:
    creator.create("Individual", list, fitness=creator.FitnessMin)

In [87]:
def _clamp(x, lo, hi):
    return lo if x < lo else hi if x > hi else x

In [88]:
class GAOptimizer:
    def __init__(
        self,
        obj,
        T_bounds=(10, 200),
        alpha_bounds=(1e-3, 1.0),
        eta_bounds=(1e-3, 1.0),
        *,
        log_space=True,
        seed=42,
        cxpb=0.9,
        mutpb=0.2,
        tournsize=3,
        elite=2,
        sigma_log=0.25,
        dT=5
    ):
        self.obj = obj
        self.Tb = T_bounds
        self.ab = alpha_bounds
        self.eb = eta_bounds
        self.log = log_space
        self.seed = int(seed)
        self.cxpb = cxpb
        self.mutpb = mutpb
        self.tournsize = tournsize
        self.elite = elite
        self.sigma_log = sigma_log
        self.dT = int(dT)
        self.toolbox = base.Toolbox()
        if self.log:
            self._ab_log = (math.log10(self.ab[0]), math.log10(self.ab[1]))
            self._eb_log = (math.log10(self.eb[0]), math.log10(self.eb[1]))
        else:
            self._ab_log = self.ab
            self._eb_log = self.eb
        random.seed(self.seed)
        np.random.seed(self.seed)
        self.toolbox.register("attr_T", random.randint, self.Tb[0], self.Tb[1])
        self.toolbox.register("attr_a", random.uniform, self._ab_log[0], self._ab_log[1])
        self.toolbox.register("attr_e", random.uniform, self._eb_log[0], self._eb_log[1])
        self.toolbox.register("individual", tools.initCycle, creator.Individual, (self.toolbox.attr_T, self.toolbox.attr_a, self.toolbox.attr_e), n=1)
        self.toolbox.register("population", tools.initRepeat, list, self.toolbox.individual)
        self.toolbox.register("evaluate", self._evaluate)
        self.toolbox.register("select", tools.selTournament, tournsize=self.tournsize)

    def _decode(self, ind):
        T = int(round(ind[0]))
        T = _clamp(T, self.Tb[0], self.Tb[1])
        if self.log:
            a = 10.0 ** _clamp(ind[1], self._ab_log[0], self._ab_log[1])
            e = 10.0 ** _clamp(ind[2], self._eb_log[0], self._eb_log[1])
        else:
            a = _clamp(ind[1], self.ab[0], self.ab[1])
            e = _clamp(ind[2], self.eb[0], self.eb[1])
        return T, float(a), float(e)

    def _evaluate(self, ind):
        T, a, e = self._decode(ind)
        try:
            v = float(self.obj(T, a, e))
        except Exception:
            v = float("inf")
        return (v,)

    def _cx(self, ind1, ind2):
        if random.random() < 0.5:
            ind1[0], ind2[0] = ind2[0], ind1[0]
        for j in (1, 2):
            g = random.random()
            a = ind1[j]
            b = ind2[j]
            ind1[j] = g * a + (1.0 - g) * b
            ind2[j] = (1.0 - g) * a + g * b
        return ind1, ind2

    def _mut(self, ind):
        if random.random() < 1.0:
            ind[0] = _clamp(int(round(ind[0] + random.randint(-self.dT, self.dT))), self.Tb[0], self.Tb[1])
        if random.random() < 1.0:
            ind[1] = _clamp(ind[1] + random.gauss(0.0, self.sigma_log), self._ab_log[0], self._ab_log[1])
        if random.random() < 1.0:
            ind[2] = _clamp(ind[2] + random.gauss(0.0, self.sigma_log), self._eb_log[0], self._eb_log[1])
        return (ind,)

    def run(self, gens=20, pop_size=24):
        pop = self.toolbox.population(n=pop_size)
        hall = tools.HallOfFame(maxsize=self.elite)
        history = []
        t0 = time.perf_counter()
        cum_time = 0.0
        for ind in pop:
            ind.fitness.values = self.toolbox.evaluate(ind)
        hall.update(pop)
        best_sofar = min(pop, key=lambda x: x.fitness.values[0])
        for g in range(gens):
            gs = time.perf_counter()
            elites = tools.selBest(pop, self.elite)
            offspring = self.toolbox.select(pop, len(pop) - self.elite)
            offspring = list(map(self.toolbox.clone, offspring))
            for i in range(0, len(offspring) - 1, 2):
                if random.random() < self.cxpb:
                    self._cx(offspring[i], offspring[i + 1])
            for i in range(len(offspring)):
                if random.random() < self.mutpb:
                    self._mut(offspring[i])
                del offspring[i].fitness.values
            invalid = [ind for ind in offspring if not ind.fitness.valid]
            for ind in invalid:
                ind.fitness.values = self.toolbox.evaluate(ind)
            pop = elites + offspring
            hall.update(pop)
            cur_best = min(pop, key=lambda x: x.fitness.values[0])
            if cur_best.fitness.values[0] < best_sofar.fitness.values[0]:
                best_sofar = cur_best
            gen_time = time.perf_counter() - gs
            cum_time = time.perf_counter() - t0
            vals = [ind.fitness.values[0] for ind in pop]
            Tb, ab, eb = self._decode(best_sofar)
            history.append({
                "iter": g,
                "best_ppl_sofar": best_sofar.fitness.values[0],
                "pop_mean": float(np.mean(vals)),
                "pop_std": float(np.std(vals)),
                "T_best": Tb,
                "alpha_best": ab,
                "eta_best": eb,
                "step_time": gen_time,
                "cum_time": cum_time
            })
        best = self._decode(best_sofar)
        best_val = best_sofar.fitness.values[0]
        final_T = best[0]
        first_hit = None
        for row in history:
            if row["T_best"] == final_T:
                first_hit = row["cum_time"]
                break
        return {
            "best": {"T": best[0], "alpha": best[1], "eta": best[2], "ppl": best_val},
            "history": history,
            "total_time": history[-1]["cum_time"] if history else 0.0,
            "time_to_best_T": first_hit if first_hit is not None else None
        }

In [89]:
class ESOptimizer:
    def __init__(
        self,
        obj,
        T_bounds=(10, 200),
        alpha_bounds=(1e-3, 1.0),
        eta_bounds=(1e-3, 1.0),
        *,
        log_space=True,
        seed=42,
        mu=12,
        lmbda=48,
        sigma_log=0.25,
        dT=5
    ):
        self.obj = obj
        self.Tb = T_bounds
        self.ab = alpha_bounds
        self.eb = eta_bounds
        self.log = log_space
        self.seed = int(seed)
        self.mu = int(mu)
        self.lmbda = int(lmbda)
        self.sigma_log = sigma_log
        self.dT = int(dT)
        self.toolbox = base.Toolbox()
        if self.log:
            self._ab_log = (math.log10(self.ab[0]), math.log10(self.ab[1]))
            self._eb_log = (math.log10(self.eb[0]), math.log10(self.eb[1]))
        else:
            self._ab_log = self.ab
            self._eb_log = self.eb
        random.seed(self.seed)
        np.random.seed(self.seed)
        self.toolbox.register("attr_T", random.randint, self.Tb[0], self.Tb[1])
        self.toolbox.register("attr_a", random.uniform, self._ab_log[0], self._ab_log[1])
        self.toolbox.register("attr_e", random.uniform, self._eb_log[0], self._eb_log[1])
        self.toolbox.register("individual", tools.initCycle, creator.Individual, (self.toolbox.attr_T, self.toolbox.attr_a, self.toolbox.attr_e), n=1)
        self.toolbox.register("population", tools.initRepeat, list, self.toolbox.individual)
        self.toolbox.register("evaluate", self._evaluate)

    def _clamp(self, ind):
        ind[0] = _clamp(int(round(ind[0])), self.Tb[0], self.Tb[1])
        ind[1] = _clamp(ind[1], self._ab_log[0], self._ab_log[1])
        ind[2] = _clamp(ind[2], self._eb_log[0], self._eb_log[1])

    def _decode(self, ind):
        T = int(round(ind[0]))
        T = _clamp(T, self.Tb[0], self.Tb[1])
        if self.log:
            a = 10.0 ** _clamp(ind[1], self._ab_log[0], self._ab_log[1])
            e = 10.0 ** _clamp(ind[2], self._eb_log[0], self._eb_log[1])
        else:
            a = _clamp(ind[1], self.ab[0], self.ab[1])
            e = _clamp(ind[2], self.eb[0], self.eb[1])
        return T, float(a), float(e)

    def _evaluate(self, ind):
        T, a, e = self._decode(ind)
        try:
            v = float(self.obj(T, a, e))
        except Exception:
            v = float("inf")
        return (v,)

    def _mut(self, parent):
        child = creator.Individual(parent[:])
        child[0] = int(round(child[0] + random.randint(-self.dT, self.dT)))
        child[1] = child[1] + random.gauss(0.0, self.sigma_log)
        child[2] = child[2] + random.gauss(0.0, self.sigma_log)
        self._clamp(child)
        return child

    def run(self, steps=24):
        parents = self.toolbox.population(n=self.mu)
        history = []
        t0 = time.perf_counter()
        cum_time = 0.0
        for ind in parents:
            ind.fitness.values = self.toolbox.evaluate(ind)
        best_sofar = min(parents, key=lambda x: x.fitness.values[0])
        for s in range(steps):
            gs = time.perf_counter()
            off = []
            for _ in range(self.lmbda):
                p = random.choice(parents)
                c = self._mut(p)
                c.fitness.values = self.toolbox.evaluate(c)
                off.append(c)
            pool = parents + off
            pool.sort(key=lambda x: x.fitness.values[0])
            parents = [creator.Individual(ind[:]) for ind in pool[:self.mu]]
            for i in range(self.mu):
                parents[i].fitness.values = pool[i].fitness.values
            cur_best = parents[0]
            if cur_best.fitness.values[0] < best_sofar.fitness.values[0]:
                best_sofar = cur_best
            step_time = time.perf_counter() - gs
            cum_time = time.perf_counter() - t0
            vals = [ind.fitness.values[0] for ind in parents]
            Tb, ab, eb = self._decode(best_sofar)
            history.append({
                "iter": s,
                "best_ppl_sofar": best_sofar.fitness.values[0],
                "pop_mean": float(np.mean(vals)),
                "pop_std": float(np.std(vals)),
                "T_best": Tb,
                "alpha_best": ab,
                "eta_best": eb,
                "step_time": step_time,
                "cum_time": cum_time
            })
        best = self._decode(best_sofar)
        best_val = best_sofar.fitness.values[0]
        final_T = best[0]
        first_hit = None
        for row in history:
            if row["T_best"] == final_T:
                first_hit = row["cum_time"]
                break
        return {
            "best": {"T": best[0], "alpha": best[1], "eta": best[2], "ppl": best_val},
            "history": history,
            "total_time": history[-1]["cum_time"] if history else 0.0,
            "time_to_best_T": first_hit if first_hit is not None else None
        }

In [92]:
import os
import csv
import json
import time
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import LatentDirichletAllocation

EVAL_CACHE = {}

def _doc_perplexities(lda, X, batch_size=1024, eps=1e-300):
    phi = lda.components_.astype(np.float64)
    phi /= phi.sum(axis=1, keepdims=True)
    n = X.shape[0]
    out = np.empty(n, dtype=np.float64)
    for s in range(0, n, batch_size):
        e = min(n, s + batch_size)
        Xb = X[s:e]
        theta = lda.transform(Xb)
        theta = np.clip(theta, 1e-12, None)
        for i in range(Xb.shape[0]):
            row = Xb[i]
            idx = row.indices
            dat = row.data
            if dat.size == 0:
                out[s + i] = 1.0
                continue
            p = theta[i].dot(phi[:, idx])
            p = np.clip(p, eps, None)
            ll = float((np.log(p) * dat).sum())
            cnt = float(dat.sum())
            out[s + i] = math.exp(-ll / max(cnt, 1.0))
    return out

def _fit_eval_full(T, alpha, eta, seed=42, max_iter=400, batch_size=2048, learning_method="online"):
    key = (int(T), float(alpha), float(eta), int(seed), int(max_iter), int(batch_size), learning_method)
    if key in EVAL_CACHE:
        return EVAL_CACHE[key]
    lda = LatentDirichletAllocation(
        n_components=int(T),
        doc_topic_prior=float(alpha),
        topic_word_prior=float(eta),
        learning_method=learning_method,
        max_iter=int(max_iter),
        batch_size=int(batch_size),
        random_state=int(seed),
        evaluate_every=-1,
        n_jobs=-1
    )
    t0 = time.perf_counter()
    lda.fit(Xtr)
    fit_time = time.perf_counter() - t0
    t1 = time.perf_counter()
    corpus_ppl = float(lda.perplexity(Xva))
    doc_ppl = _doc_perplexities(lda, Xva, batch_size=min(1024, Xva.shape[0]))
    eval_time = time.perf_counter() - t1
    res = {
        "T": int(T),
        "alpha": float(alpha),
        "eta": float(eta),
        "corpus_ppl": corpus_ppl,
        "doc_ppl_mean": float(np.mean(doc_ppl)),
        "doc_ppl_max": float(np.max(doc_ppl)),
        "fit_time": fit_time,
        "eval_time": eval_time,
        "n_iter_lda": getattr(lda, "n_iter_", None)
    }
    EVAL_CACHE[key] = res
    return res

def make_objective(seed=42, max_iter=400, batch_size=2048, learning_method="online"):
    def objective(T, a, e):
        r = _fit_eval_full(T, a, e, seed=seed, max_iter=max_iter, batch_size=batch_size, learning_method=learning_method)
        return r["corpus_ppl"]
    return objective

def _ensure_dir(p):
    os.makedirs(p, exist_ok=True)

def _write_history_csv(history_rows, path):
    fields = ["iter","best_corpus_ppl","best_doc_ppl_max","pop_mean","pop_std","T_best","alpha_best","eta_best","step_time","cum_time"]
    with open(path, "w", newline="") as f:
        w = csv.DictWriter(f, fieldnames=fields)
        w.writeheader()
        for row in history_rows:
            w.writerow(row)

def _plot_series(xs, ys, xlabel, ylabel, title, path):
    plt.figure(figsize=(7,4))
    plt.plot(xs, ys, marker="o", linewidth=1.5)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(path, dpi=150)
    plt.close()

def run_ga_with_logging(
    outdir,
    gens=24,
    pop_size=24,
    T_bounds=(10,200),
    alpha_bounds=(1e-3,1.0),
    eta_bounds=(1e-3,1.0),
    seed=42,
    max_iter=400,
    batch_size=2048,
    learning_method="online",
    cxpb=0.9,
    mutpb=0.2,
    tournsize=3,
    elite=2,
    sigma_log=0.25,
    dT=5
):
    _ensure_dir(outdir)
    obj = make_objective(seed=seed, max_iter=max_iter, batch_size=batch_size, learning_method=learning_method)
    ga = GAOptimizer(
        obj,
        T_bounds=T_bounds,
        alpha_bounds=alpha_bounds,
        eta_bounds=eta_bounds,
        log_space=True,
        seed=seed,
        cxpb=cxpb,
        mutpb=mutpb,
        tournsize=tournsize,
        elite=elite,
        sigma_log=sigma_log,
        dT=dT
    )
    res = ga.run(gens=gens, pop_size=pop_size)
    hist = []
    for row in res["history"]:
        T = row["T_best"]
        a = row["alpha_best"]
        e = row["eta_best"]
        r = _fit_eval_full(T, a, e, seed=seed, max_iter=max_iter, batch_size=batch_size, learning_method=learning_method)
        hist.append({
            "iter": row["iter"],
            "best_corpus_ppl": float(r["corpus_ppl"]),
            "best_doc_ppl_max": float(r["doc_ppl_max"]),
            "pop_mean": row["pop_mean"],
            "pop_std": row["pop_std"],
            "T_best": int(T),
            "alpha_best": float(a),
            "eta_best": float(e),
            "step_time": row["step_time"],
            "cum_time": row["cum_time"]
        })
    _write_history_csv(hist, os.path.join(outdir, "history.csv"))
    xs = [h["iter"] for h in hist]
    ys_mean = [h["best_corpus_ppl"] for h in hist]
    ys_max = [h["best_doc_ppl_max"] for h in hist]
    _plot_series(xs, ys_mean, "iter", "perplexity", "GA: mean perplexity vs iter", os.path.join(outdir, "mean_ppl.png"))
    _plot_series(xs, ys_max, "iter", "perplexity", "GA: max doc perplexity vs iter", os.path.join(outdir, "max_ppl.png"))
    avg_step_time = float(np.mean([h["step_time"] for h in hist])) if hist else 0.0
    summary = {
        "best": res["best"],
        "avg_step_time": avg_step_time,
        "total_time": res["total_time"],
        "time_to_best_T": res["time_to_best_T"]
    }
    with open(os.path.join(outdir, "summary.json"), "w") as f:
        json.dump(summary, f, indent=2)
    return {"history": hist, "summary": summary}

def run_es_with_logging(
    outdir,
    steps=24,
    T_bounds=(10,200),
    alpha_bounds=(1e-3,1.0),
    eta_bounds=(1e-3,1.0),
    seed=42,
    max_iter=400,
    batch_size=2048,
    learning_method="online",
    mu=12,
    lmbda=48,
    sigma_log=0.25,
    dT=5
):
    _ensure_dir(outdir)
    obj = make_objective(seed=seed, max_iter=max_iter, batch_size=batch_size, learning_method=learning_method)
    es = ESOptimizer(
        obj,
        T_bounds=T_bounds,
        alpha_bounds=alpha_bounds,
        eta_bounds=eta_bounds,
        log_space=True,
        seed=seed,
        mu=mu,
        lmbda=lmbda,
        sigma_log=sigma_log,
        dT=dT
    )
    res = es.run(steps=steps)
    hist = []
    for row in res["history"]:
        T = row["T_best"]
        a = row["alpha_best"]
        e = row["eta_best"]
        r = _fit_eval_full(T, a, e, seed=seed, max_iter=max_iter, batch_size=batch_size, learning_method=learning_method)
        hist.append({
            "iter": row["iter"],
            "best_corpus_ppl": float(r["corpus_ppl"]),
            "best_doc_ppl_max": float(r["doc_ppl_max"]),
            "pop_mean": row["pop_mean"],
            "pop_std": row["pop_std"],
            "T_best": int(T),
            "alpha_best": float(a),
            "eta_best": float(e),
            "step_time": row["step_time"],
            "cum_time": row["cum_time"]
        })
    _write_history_csv(hist, os.path.join(outdir, "history.csv"))
    xs = [h["iter"] for h in hist]
    ys_mean = [h["best_corpus_ppl"] for h in hist]
    ys_max = [h["best_doc_ppl_max"] for h in hist]
    _plot_series(xs, ys_mean, "iter", "perplexity", "ES: mean perplexity vs iter", os.path.join(outdir, "mean_ppl.png"))
    _plot_series(xs, ys_max, "iter", "perplexity", "ES: max doc perplexity vs iter", os.path.join(outdir, "max_ppl.png"))
    avg_step_time = float(np.mean([h["step_time"] for h in hist])) if hist else 0.0
    summary = {
        "best": res["best"],
        "avg_step_time": avg_step_time,
        "total_time": res["total_time"],
        "time_to_best_T": res["time_to_best_T"]
    }
    with open(os.path.join(outdir, "summary.json"), "w") as f:
        json.dump(summary, f, indent=2)
    return {"history": hist, "summary": summary}


In [ ]:
BASE_DIR = "runs/agnews"

ga_out = run_ga_with_logging(
    outdir=f"{BASE_DIR}/ga",
    gens=24,
    pop_size=24,
    seed=42,
    max_iter=400,
    batch_size=2048,
    learning_method="online"
)

es_out = run_es_with_logging(
    outdir=f"{BASE_DIR}/es",
    steps=24,
    seed=42,
    max_iter=400,
    batch_size=2048,
    learning_method="online"
)

print("GA summary:", ga_out["summary"])
print("ES summary:", es_out["summary"])